In [1]:
from fastlane_bot import Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T
from fastlane_bot.helpers import TxHelper
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonBot))
from fastlane_bot.testing import *
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("2.0", __VERSION__)

Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
ConstantProductCurve v2.10.1 (07/May/2023)
CarbonBot v3-b2.1 (03/May/2023)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require


C:\Users\Kveen\AppData\Local\Temp\ipykernel_35020\1582963263.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


Version = 3-b2.1 [requirements >= 2.0 is met]


# Testing the _run functions on MAINNET [NBTest021]

## MAINNET Configuration

### Set up the bot and curves

In [2]:
C = Config.new(config=Config.CONFIG_MAINNET)
assert C.DATABASE == C.DATABASE_POSTGRES
# assert C.POSTGRES_DB == "defaultdb"
assert C.NETWORK == C.NETWORK_MAINNET
assert C.PROVIDER == C.PROVIDER_ALCHEMY
bot = CarbonBot(ConfigObj=C)
assert str(type(bot.db)) == "<class 'fastlane_bot.db.manager.DatabaseManager'>"

In [3]:
CCm = bot.get_curves()
exch = {c.P("exchange") for c in CCm}
print("Number of curves:", len(CCm))
print("Number of tokens:", len(CCm.tokens()))
print("Exchanges:", exch)

Number of curves: 643
Number of tokens: 370
Exchanges: {'sushiswap_v2', 'uniswap_v2', 'carbon_v1', 'bancor_v3', 'bancor_v2'}


In [15]:
bad_curves = ['2381976568446569244243622252022377480199-0',
              '3743106036130323098097120681749450326030-0',
              '4423670769972200025023869896612986748950-0']
CCm2 = CPCContainer([x for x in CCm if x.params.exchange not in ['bancor_v2']])
len(CCm2)

549

## Actual run

In [5]:
flt = [T.USDC, T.USDT, T.ETH, T.LINK, T.BNT, T.WBTC, T.DAI]
# bot._run(flashloan_tokens=flt, CCm=CCm2)

## Analytics

In [17]:
ops = bot._find_arbitrage_opportunities_carbon_single_pairwise(flashloan_tokens=flt, CCm=CCm2, result=bot.AO_CANDIDATES)
ops

C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, t

[(Decimal('370.6943733035586774349212646484375'),
                                                   DAI-1d0F      BNT-FF1C
  732                                          5.580923e+03 -11871.346648
  12930729942995661611608235082407192035404-0 -5.580923e+03  11500.652275
  AMMIn                                        5.580923e+03  11500.652275
  AMMOut                                      -5.580923e+03 -11871.346648
  TOTAL NET                                   -1.683948e-07   -370.694373,
  [{'cid': '732',
    'tknin': 'DAI-1d0F',
    'amtin': 5580.923325454816,
    'tknout': 'BNT-FF1C',
    'amtout': -11871.3466479294,
    'error': None},
   {'cid': '12930729942995661611608235082407192035404-0',
    'tknin': 'BNT-FF1C',
    'amtin': 11500.652274625842,
    'tknout': 'DAI-1d0F',
    'amtout': -5580.923325623211,
    'error': None}],
  'BNT-FF1C',
  (CPCArbOptimizer.TradeInstruction(cid='732', tknin='DAI-1d0F', amtin=5580.923325454816, tknout='BNT-FF1C', amtout=-11871.3466479294, error

In [7]:
# ops = bot._find_arbitrage_opportunities(flashloan_tokens=flt, CCm=CCm, result=bot.AO_CANDIDATES)
# ops

In [20]:
candf = pd.DataFrame(ops).sort_values(by=0, ascending=False)
candf.reset_index(inplace=True, drop=True)
candf.iloc[0][2]

[{'cid': '2b3bf6093e55415785a319a6f48bfe9f',
  'tknin': 'WETH-6Cc2',
  'amtin': 0.9999999941928763,
  'tknout': 'DAI-1d0F',
  'amtout': -1851.4622986526228,
  'error': None},
 {'cid': '340282366920938463463374607431768211527-1',
  'tknin': 'DAI-1d0F',
  'amtin': 1.0,
  'tknout': 'WETH-6Cc2',
  'amtout': -1.0,
  'error': None}]

In [21]:
print(CCm.bycid('340282366920938463463374607431768211527-1').description)

tknx = 0 [virtual: 9999999.993761728] DAI-1d0F
tkny = 1.0 [virtual: 10000000.993761724] WETH-6Cc2
p    = 1.0000000999999996 [min=0.9999999000000096, max=1.0000000999999996] WETH-6Cc2 per DAI-1d0F
fee  = 0.002, cid = 340282366920938463463374607431768211527-1, descr = carbon_v1 ETH-EEeE/DAI-1d0F None


In [22]:
candf.iloc[0][0]

Decimal('3942.62316799607766410809893128487444987300249502060189854546479182317852973938')

In [23]:
route_struct, flashloan_amount, flashloan_token_address = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ROUTE)

C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
C:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, t

trade_instruction.cid: 340282366920938463463374607431768211527
trade_instruction.cid: 2b3bf6093e55415785a319a6f48bfe9f


In [24]:
route_struct, flashloan_amount, flashloan_token_address

([{'exchangeId': 6,
   'targetToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
   'minTargetAmount': 1,
   'deadline': 1683531370,
   'customAddress': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'customInt': 2000,
   'customData': '0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000001000000000000000000000000000000470000000000000000000000000000000000000000000000000db9ab5ae14f1fff'},
  {'exchangeId': 5,
   'targetToken': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
   'minTargetAmount': 1,
   'deadline': 1683531370,
   'customAddress': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
   'customInt': 3000,
   'customData': '0x'}],
 999000000000000000,
 '0x6B175474E89094C44Da98b954EedeAC495271d0F')

# Tenderly Tx Submission

In [13]:
### This to manually submit
bot._validate_and_submit_transaction_tenderly(
        ConfigObj = C,
        route_struct = route_struct,
        src_amount = flashloan_amount,
        src_address = flashloan_token_address,
            )

In [14]:
break

SyntaxError: 'break' outside loop (370545404.py, line 1)

# MAINNNET Preferred Tx Submission Method

In [ ]:
# from fastlane_bot.helpers import TxHelpers

In [ ]:
# pool = (
#     bot.db.get_pool(exchange_name=bot.ConfigObj.BANCOR_V3_NAME, pair_name='BNT-FF1C/ETH-EEeE')
# )
# bnt_eth = (int(pool.tkn0_balance), int(pool.tkn1_balance))

# # Init TxHelpers
# tx_helpers = TxHelpers(ConfigObj=bot.ConfigObj)
# # Submit tx



In [ ]:
# tx_helpers.validate_and_submit_transaction(route_struct=route_struct, src_amt=flashloan_amount,
#                                                     src_address=flashloan_token_address, bnt_eth=bnt_eth,
#                                                     expected_profit=Decimal('1000'), safety_override=True)

# MAINNET Forced Tx Submission Method

In [ ]:
# Initialize tx helper
tx_helper = TxHelper(
    usd_gas_limit=55,
    ConfigObj=bot.ConfigObj,
    gas_price_multiplier = 1
)
# # Submit the transaction
# tx_helper.submit_flashloan_arb_tx(
#     arb_data=route_struct,
#     flashloan_token_address=flashloan_token_address,
#     flashloan_amount=flashloan_amount,
#     verbose=True,
# )

In [ ]:
tx_helper.gas_limit